In [10]:
import sympy as sp
import operator
import itertools
from functools import partial, reduce
from lbmpy.chapman_enskog import *
from lbmpy.creationfunctions import createLatticeBoltzmannMethod
from pystencils.display_utils import toDot
from pystencils.sympyextensions import productSymmetric
from IPython.display import display

from lbmpy.chapman_enskog.chapman_enskog import *

DiffOp = DiffOperator

sp.init_printing()

c = sp.Matrix([expandedSymbol("c", subscript=i) for i in range(3)])

In [39]:
method = createLatticeBoltzmannMethod(stencil="D2Q9", 
                                      equilibriumAccuracyOrder=3,
                                      compressible=True,
                                      useContinuousMaxwellianEquilibrium=True,
                                      c_s_sq=sp.Symbol("c_s")**2,
                                      #relaxationRate=1 / sp.Symbol("tau")
                                      method='trt'
                                     )
method

Moment,Eq. Value,Relaxation Rate
$1$,$\rho$,$\omega_{0}$
$y$,$\rho u_{1}$,$\omega_{1}$
$y^{2}$,$c_{s}^{2} \rho + \rho u_{1}^{2}$,$\omega_{0}$
$x$,$\rho u_{0}$,$\omega_{1}$
$x y$,$\rho u_{0} u_{1}$,$\omega_{0}$
$x y^{2}$,$c_{s}^{2} \rho u_{0} + \rho u_{0} u_{1}^{2}$,$\omega_{1}$
$x^{2}$,$c_{s}^{2} \rho + \rho u_{0}^{2}$,$\omega_{0}$
$x^{2} y$,$c_{s}^{2} \rho u_{1} + \rho u_{0}^{2} u_{1}$,$\omega_{1}$
$x^{2} y^{2}$,$c_{s}^{4} \rho + c_{s}^{2} \rho u_{0}^{2} + c_{s}^{2} \rho u_{1}^{2}$,$\omega_{0}$


In [40]:
eq = getTaylorExpandedLbEquation(dim=method.dim)
eq

      Δₜ⋅c_{0}⋅Diff(Cf, 0, -1)   Δₜ⋅c_{1}⋅Diff(Cf, 1, -1)   Δₜ⋅Diff(Cf, t, -1)
-Cf + ──────────────────────── + ──────────────────────── + ──────────────────
                 2                          2                       2         

                                                               
 + c_{0}⋅Diff(f, 0, -1) + c_{1}⋅Diff(f, 1, -1) + Diff(f, t, -1)
                                                               

In [41]:
epsDict = useChapmanEnskogAnsatz(eq)
epsDict

⎧                               Δₜ⋅c_{0}⋅Diff(Cf^{(0)}, 0, 1)   Δₜ⋅c_{1}⋅Diff(
⎨0: -Cf__{(0)}, 1: -Cf__{(1)} + ───────────────────────────── + ──────────────
⎩                                             2                               

Cf^{(0)}, 1, 1)   Δₜ⋅Diff(Cf^{(0)}, t, 1)                                     
─────────────── + ─────────────────────── + c_{0}⋅Diff(f^{(0)}, 0, 1) + c_{1}⋅
2                            2                                                

                                                           Δₜ⋅c_{0}⋅Diff(Cf^{(
Diff(f^{(0)}, 1, 1) + Diff(f^{(0)}, t, 1), 2: -Cf__{(2)} + ───────────────────
                                                                         2    

1)}, 0, 1)   Δₜ⋅c_{1}⋅Diff(Cf^{(1)}, 1, 1)   Δₜ⋅Diff(Cf^{(0)}, t, 2)   Δₜ⋅Diff
────────── + ───────────────────────────── + ─────────────────────── + ───────
                           2                            2                     

(Cf^{(1)}, t, 1)                                

In [42]:
epsDict[1] 

             Δₜ⋅c_{0}⋅Diff(Cf^{(0)}, 0, 1)   Δₜ⋅c_{1}⋅Diff(Cf^{(0)}, 1, 1)   Δ
-Cf__{(1)} + ───────────────────────────── + ───────────────────────────── + ─
                           2                               2                  

ₜ⋅Diff(Cf^{(0)}, t, 1)                                                        
────────────────────── + c_{0}⋅Diff(f^{(0)}, 0, 1) + c_{1}⋅Diff(f^{(0)}, 1, 1)
          2                                                                   

                      
 + Diff(f^{(0)}, t, 1)
                      

In [43]:
momEq = takeMomentsG(epsDict[1])
momEq

            Δₜ⋅Diff(\Upsilon, 0, 1)   Δₜ⋅Diff(\Upsilon, 1, 1)   Δₜ⋅Diff(\Upsil
-\Upsilon + ─────────────────────── + ─────────────────────── + ──────────────
                       2                         2                         2  

on, t, 1)                                                      
───────── + Diff(\Pi, 0, 1) + Diff(\Pi, 1, 1) + Diff(\Pi, t, 1)
                                                               

In [51]:
methodMoments = LbMethodEqMoments(method)
momEq = substituteCollisionOperatorMoments(momEq, method)
momEq

          Δₜ⋅Diff(rho*u_0, 0, 1)                                              
-\Pi⋅ω₀ + ────────────────────── + Diff(rho*u_0, 0, 1) + Diff(c_s**2*rho + rho
                    2                                                         

                                                            
*u_0**2, t, 1) + Diff(c_s**2*rho*u_1 + rho*u_0**2*u_1, 1, 1)
                                                            

In [52]:
insertMoments(momEq, methodMoments).expand()

          Δₜ⋅Diff(rho*u_0, 0, 1)                                              
-\Pi⋅ω₀ + ────────────────────── + Diff(rho*u_0, 0, 1) + Diff(c_s**2*rho + rho
                    2                                                         

                                                            
*u_0**2, t, 1) + Diff(c_s**2*rho*u_1 + rho*u_0**2*u_1, 1, 1)
                                                            

In [53]:
momEq = takeMomentsG(epsDict[2] * c[0])
momEq = substituteCollisionOperatorMoments(momEq, method)
momEq = insertMoments(momEq, methodMoments).expand()
func = [sp.Symbol("rho")] + list(c) + list(momEq.atoms(CeMoment))
#momEq = expandUsingLinearity(momEq, func).expand()
momEq

Δₜ⋅Diff(\Pi*omega_0, 0, 1)   Δₜ⋅Diff(\Pi*omega_0, 1, 1)                       
────────────────────────── + ────────────────────────── + Diff(\Pi, 0, 1) + Di
            2                            2                                    

                                   
ff(\Pi, 1, 1) + Diff(rho*u_0, t, 2)
                                   

In [54]:
momEq = takeMomentsG(epsDict[1] * c[0] * c[0])
momEq = substituteCollisionOperatorMoments(momEq, method)
momEq = insertMoments(momEq, methodMoments).expand()
func = [sp.Symbol("rho")] + list(c) + list(momEq.atoms(CeMoment))
#momEq = expandUsingLinearity(momEq, func).expand()
momEq

/local/bauer/code/lbmpy/lbmpy/chapman_enskog/chapman_enskog.py:130: UserWarning: Could not replace postcollision moment x**3, not found in methods relaxation info
  "not found in methods relaxation info")


          Δₜ⋅Diff(rho*u_0, 0, 1)                                              
-\Pi⋅ω₀ + ────────────────────── + Diff(rho*u_0, 0, 1) + Diff(c_s**2*rho + rho
                    2                                                         

                                                            
*u_0**2, t, 1) + Diff(c_s**2*rho*u_1 + rho*u_0**2*u_1, 1, 1)
                                                            

--> x**3 works just like x